# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/dhruv04/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dhruv04/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dhruv04/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df = pd.read_csv("IMDB Dataset.csv")
rev = df['review']
sent = df['sentiment']

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

rev1 = []

# Removing html tags and URLs
for i in range(len(rev)):
    soup = BeautifulSoup(rev[i], 'html.parser')
    for a in soup.findAll('a', href=True):
        a.extract()
    text = soup.get_text()
    rev1.append(text)

words = []
stop_words = set(stopwords.words('english'))
for i in range(len(rev1)):
    words.append(nltk.word_tokenize(rev1[i]))
    words[i] = [word.lower() for word in words[i] if word.isalnum()] #Removing non alphanumeric characters
    words[i] = [word.lower() for word in words[i] if word not in stop_words] #Removing Stopwords
    # print(len(words[i]))


# #Function for stemming each review
# def stemSentence(sentence):
#     stem_sentence=""
#     for word in sentence:
#         stem_sentence = ' ' + (porter.stem(word))
#     return stem_sentence

# #Function for Lematizing each review
# def lemSentence(sentence):
#     lem_sentence = ""
#     for word in sentence:
#         lem_sentence = lem_sentence+' '+(lemmatizer.lemmatize(word))
#     return lem_sentence


In [4]:
# Performing Stemming

porter = PorterStemmer()
words2 = []
for i in range(len(words)):
    tstr = ""
    for j in range(len(words[i])):
        words[i][j] = porter.stem(words[i][j])  
        tstr = tstr + ' ' + words[i][j]
    words2.append(tstr)


In [5]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
avg = 0
pos = 0
neg = 0
Max = 0
for i in range(len(words)):
    avg+=len(words[i])
    if(sent[i] == 'positive'):
        pos+=1
    else:
        neg+=1
    if(len(words[i]) > Max):
        Max = len(words[i])
avg = avg/len(words)
print("Average length of sentence: ", avg)
print("Positive reviews: ", pos)
print("Negative reviews: ", neg)
print("Maximum Length of sentences: ", Max)
print("Reviews after stemming:")
print(words2[:5])


Average length of sentence:  113.89472
Positive reviews:  25000
Negative reviews:  25000
Maximum Length of sentences:  1404
Reviews after stemming:
[' one review mention watch 1 oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far would say main appeal show due fact goe show would dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal could say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well ma

# Naive Bayes classifier

In [6]:
# get reviews column from df
reviews = words2

# get labels column from df
labels = sent

In [7]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print(encoded_labels)
# print(labels)
classes = encoder.classes_
print(classes)

[1 1 1 ... 0 0 0]
['negative' 'positive']


In [8]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.20) 
# print(train_sentences.shape, train_labels.shape)
print(len(test_sentences), len(train_sentences))


10000 40000


Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
X = X.toarray()
vocab = vec.get_feature_names()
print(len(vocab))

3000


In [10]:
# Use laplace smoothing for words in test set not present in vocab of train set
def lapSmoothing():
    ans = 1/6000
    return ans


In [25]:
# Build the model. Don't use the model from sklearn
pos_cnt = [0]*3000
neg_cnt = [0]*3000
print ("---------------- Training In Progress --------------------")
for i in range(len(train_labels)):
    if(train_labels[i] == 0):
        neg_cnt = neg_cnt+X[i]
    else:
        pos_cnt = pos_cnt+X[i]
print ("---------------- Training Completed --------------------")
# print(pos_cnt)
# print(neg_cnt)

---------------- Training In Progress --------------------
---------------- Training Completed --------------------


In [27]:
# Test the model on test set and report Accuracy

vec2 = CountVectorizer(max_features=3000)
Y = vec2.fit_transform(test_sentences)
Y = Y.toarray()
vocab_test = vec2.get_feature_names()
pos_prob=[0]*10000
neg_prob=[0]*10000
pred=[0]*10000
for i in range(len(test_sentences)):
    for j in range(len(vocab_test)):
        if(Y[i][j]>0):
            if(vocab_test[j] in vocab):
                pos_prob[i]=pos_prob[i]+(pos_cnt[vocab.index(vocab_test[j])]+1)/6000
                neg_prob[i]=neg_prob[i]+(neg_cnt[vocab.index(vocab_test[j])]+1)/6000
            else:
                pos_prob[i]=(pos_prob[i]*6000 + 1)/6000
                neg_prob[i]=(neg_prob[i]*6000 + 1)/6000
    if(pos_prob[i]>=neg_prob[i]):
        pred[i]=1

acc=0
for i in range(len(pred)):
    if(pred[i]==test_labels[i]):
        acc=acc+1

print("Result of Naive Bayes Model:")
print("correct predictions = ",acc)
print("wrong predictions = ",len(pred)-acc)
print("Test Set Accuracy = ", acc/100,"%")


Result of Naive Bayes Model:
correct predictions =  7279
wrong predictions =  2721
Test Set Accuracy =  72.79 %


# *LSTM* based Classifier

Use the above train and test splits.

In [10]:
# Hyperparameters of the model
vocab_size = vocab # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [11]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [12]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          77000     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 164,601
Trainable params: 164,601
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 5

history = model.fit(train_padded, train_labels, 
                    epochs= num_epochs, verbose=1, 
                    validation_split= 0.1)

Epoch 1/5
1125/1125 [==============================] - 160s 138ms/step - loss: 0.4453 - accuracy: 0.7928 - val_loss: 0.3503 - val_accuracy: 0.8495
Epoch 2/5
1125/1125 [==============================] - 162s 144ms/step - loss: 0.3646 - accuracy: 0.8455 - val_loss: 0.3647 - val_accuracy: 0.8570
Epoch 3/5
1125/1125 [==============================] - 149s 133ms/step - loss: 0.3428 - accuracy: 0.8551 - val_loss: 0.3458 - val_accuracy: 0.8515
Epoch 4/5
1125/1125 [==============================] - 147s 131ms/step - loss: 0.3332 - accuracy: 0.8595 - val_loss: 0.3471 - val_accuracy: 0.8500
Epoch 5/5
1125/1125 [==============================] - 167s 149ms/step - loss: 0.3171 - accuracy: 0.8674 - val_loss: 0.3429 - val_accuracy: 0.8535


In [17]:
# Calculate accuracy on Test data
model.evaluate(test_padded, test_labels)
prediction = model.predict(test_padded)

# Get probabilities
prob = []
for i in range(len(prediction)):
    prob.append(prediction[i][0])

# Get labels based on probability 1 if p>= 0.5 else 0
prob_labels = []
for i in range(len(prob)):
    if(prob[i]>=0.5):
        prob_labels.append(1)
    else:
        prob_labels.append(0)

# Accuracy : one can use classification_report from sklearn
target_names = ['negative', 'positive']
print(classification_report(test_labels, prob_labels,target_names=target_names))

313/313 [==============================] - 12s 36ms/step - loss: 0.3495 - accuracy: 0.8552
              precision    recall  f1-score   support

    negative       0.86      0.85      0.85      4957
    positive       0.85      0.86      0.86      5043

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## Get predictions for random examples

In [16]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
print("Probabilities:")
print(model.predict(padded))
sentiments = model.predict(padded)
sent_labels = []
# Get labels based on probability 1 if p>= 0.5 else 0
for i in range(3):
    if(sentiments[i][0]>=0.5):
        sent_labels.append(1)
    else:
        sent_labels.append(0)
for i in range(3):
    if(sent_labels[i] == 1):
        print('positive')
    else:
        print('negative')



Probabilities:
[[0.66182697]
 [0.16814485]
 [0.14222002]]
positive
negative
negative
